# Lifei Wang 
## 8855277
## CSCN8000-Assignment2

Download heart disease dataset heart.csv in Resources folder and do following,  https://www.kaggle.com/fedesoriano/heart-failure-prediction

1. Consider the heart disease dataset in pandas dataframe

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('./heart.csv')
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [3]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


2. Remove outliers using mean,median,Z score. 

In [4]:
num_col = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
df_mean = df
# use mean
for col in num_col:
    mean = df_mean[col].mean()
    std = df_mean[col].std()
    lower_bound = mean-2*std
    upper_bound = mean+2*std
    df_mean = df_mean[(df_mean[col] >= lower_bound) & (df_mean[col] <= upper_bound)]
df_mean.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000
mean,53.322709,130.679947,199.423639,0.219124,137.851262,0.737450,0.519256
std,8.634772,14.651895,102.426517,0.413927,23.463568,0.832536,0.499961
min,35.000000,96.000000,0.000000,0.000000,87.000000,-1.100000,0.000000
25%,47.000000,120.000000,177.000000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.400000,1.000000
75%,60.000000,140.000000,266.000000,0.000000,156.000000,1.500000,1.000000
max,72.000000,165.000000,412.000000,1.000000,186.000000,2.900000,1.000000


In [5]:
# use median
df_median = df
for col in num_col:
    median = df_median[col].median()
    median_absd = np.median(np.abs(df_median[col]-median))
    lower_bound = median-2*median_absd
    upper_bound = median+2*median_absd
    df_median = df_median[(df_median[col] >= lower_bound) & (df_median[col] <= upper_bound)]
df_median.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,307.000000,307.000000,307.000000,307.000000,307.000000,307.000000,307.000000
mean,52.485342,129.657980,233.745928,0.149837,143.065147,0.410098,0.361564
std,7.124634,11.319064,36.791329,0.357494,18.430287,0.540149,0.481238
min,40.000000,110.000000,149.000000,0.000000,105.000000,0.000000,0.000000
25%,47.000000,120.000000,206.000000,0.000000,129.000000,0.000000,0.000000
50%,53.000000,130.000000,233.000000,0.000000,142.000000,0.000000,0.000000
75%,58.000000,140.000000,263.000000,0.000000,159.500000,1.000000,1.000000
max,68.000000,150.000000,307.000000,1.000000,176.000000,1.500000,1.000000


In [6]:
# use z-score
df_z_score = df
for col in num_col:
    z_score = (df_z_score[col] - df_z_score[col].mean()) / df_z_score[col].std()
    df_z_score = df_z_score[abs(z_score) < 2]
df_z_score.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000
mean,53.322709,130.679947,199.423639,0.219124,137.851262,0.737450,0.519256
std,8.634772,14.651895,102.426517,0.413927,23.463568,0.832536,0.499961
min,35.000000,96.000000,0.000000,0.000000,87.000000,-1.100000,0.000000
25%,47.000000,120.000000,177.000000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.400000,1.000000
75%,60.000000,140.000000,266.000000,0.000000,156.000000,1.500000,1.000000
max,72.000000,165.000000,412.000000,1.000000,186.000000,2.900000,1.000000


Based on overall performance, remove outliers with mean value.

In [7]:
# use mean
for col in num_col:
    mean = df[col].mean()
    std = df[col].std()
    lower_bound = mean-2*std
    upper_bound = mean+2*std
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,57,F,ASY,140,241,0,Normal,123,Y,0.2,Flat,1
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


3. Convert text columns to numbers using label encoding and one hot encoding


In [8]:
convert_col = ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope', 'HeartDisease']
# label encoding
for col in convert_col:
    # represent data in categorical format
    df[col] = pd.Categorical(df[col])
    # convert categorical into numerical values
    df[col] = df[col].cat.codes

print('Label encoding:')
df

Label encoding:


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,57,0,0,140,241,0,1,123,1,0.2,1,1
913,45,1,3,110,264,0,1,132,0,1.2,1,1
915,57,1,0,130,131,0,1,115,1,1.2,1,1
916,57,0,1,130,236,0,0,174,0,0.0,1,1


In [9]:
# one hot encoding
df_onehot = df
df_onehot = pd.get_dummies(df, columns=convert_col)
df_onehot

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,Sex_0,Sex_1,ChestPainType_0,ChestPainType_1,ChestPainType_2,...,RestingECG_0,RestingECG_1,RestingECG_2,ExerciseAngina_0,ExerciseAngina_1,ST_Slope_0,ST_Slope_1,ST_Slope_2,HeartDisease_0,HeartDisease_1
0,40,140,289,172,0.0,False,True,False,True,False,...,False,True,False,True,False,False,False,True,True,False
1,49,160,180,156,1.0,True,False,False,False,True,...,False,True,False,True,False,False,True,False,False,True
2,37,130,283,98,0.0,False,True,False,True,False,...,False,False,True,True,False,False,False,True,True,False
3,48,138,214,108,1.5,True,False,True,False,False,...,False,True,False,False,True,False,True,False,False,True
4,54,150,195,122,0.0,False,True,False,False,True,...,False,True,False,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,57,140,241,123,0.2,True,False,True,False,False,...,False,True,False,False,True,False,True,False,False,True
913,45,110,264,132,1.2,False,True,False,False,False,...,False,True,False,True,False,False,True,False,False,True
915,57,130,131,115,1.2,False,True,True,False,False,...,False,True,False,False,True,False,True,False,False,True
916,57,130,236,174,0.0,True,False,False,True,False,...,True,False,False,True,False,False,True,False,False,True


4. Apply scaling

In [11]:
standard_scaler = StandardScaler()
# choose label encoding for df
# df = df_label

df[num_col] = df[num_col].astype(float)
df[num_col] = standard_scaler.fit_transform(df[num_col])
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,-1.543939,1,1,0.636522,0.875124,0,1,1.456361,0,-0.886376,2,0
1,-0.500949,0,2,2.002440,-0.189761,0,1,0.774000,0,0.315571,1,1
2,-1.891603,1,1,-0.046438,0.816506,0,2,-1.699560,0,-0.886376,2,0
3,-0.616837,0,0,0.499930,0.142405,0,1,-1.273084,1,0.916544,1,1
4,0.078490,1,2,1.319481,-0.043217,0,1,-0.676018,0,-0.886376,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,0.426153,0,0,0.636522,0.406184,0,1,-0.633371,1,-0.645987,1,1
913,-0.964500,1,3,-1.412356,0.630884,0,1,-0.249542,0,0.555960,1,1
915,0.426153,1,0,-0.046438,-0.668471,0,1,-0.974551,1,0.555960,1,1
916,0.426153,0,1,-0.046438,0.357336,0,0,1.541657,0,-0.886376,1,1


5. Build a machine learning classification model using support vector machine. 
  
   Demonstrate the standalone model as well as Bagging model and include observations about the performance

In [28]:
# split data
X = df.drop(columns=['HeartDisease'])
y = df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# build svm model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# predict on test set
y_pred = svm_model.predict(X_test)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8675496688741722


In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86        72
           1       0.86      0.89      0.87        79

    accuracy                           0.87       151
   macro avg       0.87      0.87      0.87       151
weighted avg       0.87      0.87      0.87       151



In [15]:
print(confusion_matrix(y_test, y_pred))

[[61 11]
 [ 9 70]]


In [27]:
# build bagging model with svm base estimator
bagging_model = BaggingClassifier(estimator=SVC(kernel='linear'))
bagging_model.fit(X_train, y_train)

# predict on test set
y_pred_bagging = bagging_model.predict(X_test)

# evaluate bagging model
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
accuracy_bagging

0.8609271523178808

In [20]:
print(classification_report(y_test, y_pred_bagging))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86        72
           1       0.86      0.89      0.87        79

    accuracy                           0.87       151
   macro avg       0.87      0.87      0.87       151
weighted avg       0.87      0.87      0.87       151



In [21]:
print(confusion_matrix(y_test, y_pred_bagging))

[[61 11]
 [ 9 70]]


The standalone model using SVM and bagging model has exactly the same performance.

6. Now use decision tree classifier. 
    
    Use standalone model as well as Bagging and check if you notice any difference in performance

In [23]:
# build decision tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

y_pred_dt = dt_classifier.predict(X_test)

# evaluate decision tree classifier
accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_dt

0.8410596026490066

In [24]:
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.82      0.85      0.84        72
           1       0.86      0.84      0.85        79

    accuracy                           0.84       151
   macro avg       0.84      0.84      0.84       151
weighted avg       0.84      0.84      0.84       151



In [25]:
print(confusion_matrix(y_test, y_pred_dt))

[[61 11]
 [13 66]]


In [31]:
# create bagging model with decision tree classifier
bagging_dt_model = BaggingClassifier(estimator = DecisionTreeClassifier(random_state=42), n_estimators=10, random_state=42)
bagging_dt_model.fit(X_train, y_train)

y_pred_bagging_dt = bagging_dt_model.predict(X_test)

accuracy_bagging_dt = accuracy_score(y_test, y_pred_bagging_dt)
accuracy_bagging_dt

0.8609271523178808

In [32]:
print(classification_report(y_test, y_pred_bagging_dt))

              precision    recall  f1-score   support

           0       0.85      0.86      0.86        72
           1       0.87      0.86      0.87        79

    accuracy                           0.86       151
   macro avg       0.86      0.86      0.86       151
weighted avg       0.86      0.86      0.86       151



In [33]:
print(confusion_matrix(y_test, y_pred_bagging_dt))

[[62 10]
 [11 68]]


7. Comparing performance of svm and decision tree classifier 

    figure out where it makes most sense to use bagging and why. 

Based on the output above, model with bagging has higher accuracy, precision, recall and f1-score therefore the performance is better.

Bagging is the best when there is a large dataset and complicated model such that it could improve the performance.